## Recommendation Dashboard 

### Project Description:

The following codes generates a restaurant recommendation dashboard app that allows the user to determine if he/she wants to be recommended restaurant by all standards of by food standards.

The recommendation engine is powered by a Singular Value Decomposition (SVD) algorithm. 

In [19]:
# Import packages
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_auth
import dash_table

import numpy as np
import pandas as pd
from scipy import stats
from scipy.spatial.distance import cosine
from pickle import dump
from pickle import load

import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools

# Create dashboared username and password
USERNAME_PASSWORD_PAIRS = [['data','analyst']]

# Establish dash app
app = dash.Dash(external_stylesheets = [dbc.themes.GRID])

# Initialize dashboard username and password
auth = dash_auth.BasicAuth(app,USERNAME_PASSWORD_PAIRS)
server = app.server


# Create starting table column names for restaurant reviews and ratings
doc_sf_col = ['Business','Stars','']
doc_sf_data = [{'Business':'-','Stars':'-','':'-'}]
doc_sf_data = pd.DataFrame(doc_sf_data)

# Create starting list of restaurant names (all reviews)
doc1 = pd.read_csv('Data/All.csv')
doc1 = doc1[['business_name']]
doc1.columns = ['1']
doc1 = doc1['1'].unique()

new_doc1 = []
for i in doc1:
    string = str(i)
    new_doc1.append(string)
doc1 = new_doc1
doc1.sort()
search_options = [{'label': i, 'value': i} for i in doc1]

# Create radio button options for user ratings
radio_options = [{'label': i, 'value': i} for i in [0,1,2,3,4,5]]

# Establish app layout
app.layout = html.Div([
    
    # App Header
    html.Div([html.H1('Toronto Restaurant Recommender Dashboard',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})]),
    
    # Radio buttons for all or food standards
    html.Div([dcc.RadioItems(id = 'rate',
                             options = [{'label': 'Service & Food Based', 'value': 1},
                                        {'label': 'Food Based', 'value': 2}],
                             value = 1,
                             labelStyle = {'display': 'inline-block'},
                             style = {'textAlign':'center',
                                      'padding-top':30,
                                      'font-family':'Arial, Helvetica, sans-serif'})]),
    
    # Header for choose your restaurant to rate to recieve alike recommendations 
    html.H2('Choose & Rate Your Restaurant',
            style = {'font-family':'Arial, Helvetica, sans-serif',
                     'textAlign':'center',
                     'padding-top':5,
                     'padding-bottom':5}),
    
    # Dropdown menue 1
    html.Div([dbc.Row([dbc.Col(dcc.Dropdown(id = 'search1',
                                            options = search_options,
                                            value = "Uncle Tetsu's Japanese Cheesecake",
                                            style = {'textAlign':'center'})),
                     
                       # Dropdown menue 2
                       dbc.Col(dcc.Dropdown(id = 'search2',
                                            options = search_options,
                                            value = "Kyoto House Japanese Restaurant",
                                            style = {'textAlign':'center'})),
                       
                       # Dropdown menue 3
                       dbc.Col(dcc.Dropdown(id = 'search3',
                                            options = search_options,
                                            value = "Wheat Sheaf Tavern",
                                            style = {'textAlign':'center'}))])],
             
             style = {'font-family':'Arial, Helvetica, sans-serif',
                      'padding-left':20,
                      'padding-right':20}),
              
    # Radio buttons 1     
    html.Div([dbc.Row([dbc.Col(dcc.RadioItems(id = 'rating1',
                                              options = radio_options,
                                              value = 5,
                                              labelStyle = {'display': 'inline-block'})),
                       
                       # Radio buttons 2
                       dbc.Col(dcc.RadioItems(id = 'rating2',
                                              options = radio_options,
                                              value = 5,
                                              labelStyle = {'display': 'inline-block'})),
                       
                       # Radio buttons 3
                       dbc.Col(dcc.RadioItems(id = 'rating3',
                                              options = radio_options,
                                              value = 5,
                                              labelStyle = {'display': 'inline-block'}))])],
             
             style = {'font-family':'Arial, Helvetica, sans-serif',
                      'textAlign':'center',
                      'padding-top':10}),
    
    # Horizontal bar plot with recommendation names and strength of recommendation
    html.Div([dcc.Graph('feature_graphic')],
              style = {'font-family':'Arial, Helvetica, sans-serif',
                       'padding-top':30,
                       'padding-right':120,
                       'padding-bottom':25,
                       'margin-left':50}),
    
    # App instructions
    html.Div([html.H1('Instructions',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'}),
              
              html.Div(html.P(["The dashboard displays restaurant recommendations based on your \
                               (user) ratings of three restaurants. Above the dropdown menus, you have \
                               the option of receiving all or food based standard recommendations; select \
                               according to your taste. The following dropdown \
                               menus can be leveraged either by typing or scrolling for the restaurant you visited \
                               and would like to rate. Below the dropdowns are radio buttons where \
                               you can rate your selection. Once all fields are filled out, the bar plot will \
                               fill into scaled strengths of recommendations (top 10 in descending order). \
                               The name of the recommended restaurants can be found on the y-axis of the plot. \
                               The reviews and ratings for the recommended restaurants can be viewed below."]),
                       style = {'padding':30,
                                'font-family':'Arial, Helvetica, sans-serif',
                                'line-height':30,
                                'textAlign':'center',
                                'fontSize':20})]),
    
    # Header for reviews data
    html.Div([html.H1('Reviews',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})]),
    
    # Reviews data table
    dash_table.DataTable(id = 'table',
                         style_data = {'whiteSpace': 'normal',
                                       'height': 'auto',
                                       'border': '1px solid black',
                                       'fontSize':20},
                         columns = [{"name": i, "id": i} for i in doc_sf_col],
                         data = doc_sf_data.to_dict('records'),
                         style_cell = {'textAlign':'left',
                                       'height': 'auto',
                                       'padding':20,
                                       'fontSize':18,
                                       'font-family':'Arial, Helvetica, sans-serif'},
                         style_table = {'width': '100%',
                                        'padding-bottom':0,
                                        'padding-top':0,
                                        'display':'inline-block',
                                        'maxHeight': '600px',
                                        'overflowY': 'scroll'},
                         style_header = {'backgroundColor':'rgb(230, 230, 230)',
                                         'fontWeight':'bold',
                                         'textAlign':'center',
                                         'font-family':'Arial, Helvetica, sans-serif',
                                         'border': '1px solid black',
                                         'padding':0,
                                         'line-height':2.5},
                         style_as_list_view = True),
    
    # Ending block
    html.Div([html.H1('',
                      style = {'padding':30,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})])],
    style={'margin':0})

# Function outputs and parameters
@app.callback([Output('search1','options'),
               Output('search2','options'),
               Output('search3','options')],
              [Input('rate','value')])

# App function for dropdown meanues
def update_date_dropdown(rate):

    # By use of the radio buttons for all or food based standards, specific restaurants will be generated
    # for the dropdown menues
    if rate == 1:
        # All standards
        doc1 = pd.read_csv('Data/All.csv')
        doc1 = doc1[['business_name']]
        doc1.columns = ['1']
        doc1 = doc1['1'].unique()

        new_doc1 = []
        for i in doc1:
            string = str(i)
            new_doc1.append(string)
        doc1 = new_doc1
        doc1.sort()
        search_options = [{'label': i, 'value': i} for i in doc1]
    else:
        # Food standards
        doc1 = pd.read_csv('Data/Food.csv')
        doc1 = doc1[['business_name']]
        doc1.columns = ['2']
        doc1 = doc1['2'].unique()
        
        new_doc1 = []
        for i in doc1:
            string = str(i)
            new_doc1.append(string)
        doc1 = new_doc1
        doc1.sort()
        search_options = [{'label': i, 'value': i} for i in doc1]
    
    return search_options,search_options,search_options

# Function outputs and parameters
@app.callback(Output('feature_graphic','figure'),
             [Input('rate','value'),
              Input('search1','value'),
              Input('rating1','value'),
              Input('search2','value'),
              Input('rating2','value'),
              Input('search3','value'),
              Input('rating3','value')])

# Recommendation function
def recommendation(rate, search1, rating1, search2, rating2, search3, rating3):
    
    # Create recommendation function
    def recommendation (standard, rest1, rating1, rest2, rating2, rest3, rating3):
        
        # Deciding which algorithm to use based on standard
        if standard == 1:
            algo = load(open('SVD_Model.pkl', 'rb'))
        else:
            algo = load(open('SVD_Model_Food.pkl', 'rb'))

            # Cosine distance between vectors calculation
        def cosine_distance(vector_a = np.array, vector_b = np.array):
            return cosine(vector_a, vector_b)

        # Retrieve vectors by restaurant name
        def get_vector_by_rest_name(rest_name, trained_model):
            rest_row_idx = trained_model.trainset._raw2inner_id_items[rest_name]
            return trained_model.qi[rest_row_idx]

        # Get vectors by restaurant name for three restaurants
        vector1 = get_vector_by_rest_name(rest1, algo)
        score1 = rating1
        vector2 = get_vector_by_rest_name(rest2, algo)
        score2 = rating2
        vector3 = get_vector_by_rest_name(rest3, algo)
        score3 = rating3

        ##############################################################################################################

        # Calculate cosine similarity for all three chosen restaurants' vectors against all other restaurant vectors
        similarity_table1 = []
        for rest_name in algo.trainset._raw2inner_id_items.keys():
            rest_vector = get_vector_by_rest_name(rest_name, algo)
            similarity_score = cosine_distance(vector1, rest_vector)
            similarity_table1.append((1-similarity_score, rest_name))

        # Convert similarity table into a data frame
        rest_rec1 = pd.DataFrame(similarity_table1, columns = ['similarity', 'restaurant name'])
        # Scale cosine score by rating
        rest_rec1['similarity'] = rest_rec1['similarity'] * score1
        # Sort data set to descending
        rest_rec1 = rest_rec1.sort_values('similarity', ascending = False)

        ##############################################################################################################

        # Calculate cosine similarity for all three chosen restaurants' vectors against all other restaurant vectors
        similarity_table2 = []
        for rest_name in algo.trainset._raw2inner_id_items.keys():
            rest_vector = get_vector_by_rest_name(rest_name, algo)
            similarity_score = cosine_distance(vector2, rest_vector)
            similarity_table2.append((1-similarity_score, rest_name))

        # Convert similarity table into a data frame
        rest_rec2 = pd.DataFrame(similarity_table2, columns = ['similarity', 'restaurant name'])
        # Scale cosine score by rating
        rest_rec2['similarity'] = rest_rec2['similarity'] * score2
        # Sort data set to descending
        rest_rec2 = rest_rec2.sort_values('similarity', ascending = False)

        ##############################################################################################################

        # Calculate cosine similarity for all three chosen restaurants' vectors against all other restaurant vectors
        similarity_table3 = []
        for rest_name in algo.trainset._raw2inner_id_items.keys():
            rest_vector = get_vector_by_rest_name(rest_name, algo)
            similarity_score = cosine_distance(vector3, rest_vector)
            similarity_table3.append((1-similarity_score, rest_name))

        # Convert similarity table into a data frame
        rest_rec3 = pd.DataFrame(similarity_table3, columns = ['similarity', 'restaurant name'])
        # Scale cosine score by rating
        rest_rec3['similarity'] = rest_rec3['similarity'] * score3
        # Sort data set to descending
        rest_rec3 = rest_rec3.sort_values('similarity', ascending = False).reset_index()

        ##############################################################################################################

        # Create a list of all data frames
        df_list = [rest_rec1, rest_rec2, rest_rec3]
        # Concatenate all data frames by axis 0
        rest_rec4 = pd.concat(df_list, axis = 0)
        # Remove all three chosen restaurants 
        rest_rec4 = rest_rec4.loc[(rest_rec4['restaurant name'] != rest1) & (rest_rec4['restaurant name'] != rest2) &
                                  (rest_rec4['restaurant name'] != rest3)].reset_index(drop = True)
        # Scale cosine score by duplicates
        rest_rec4 = rest_rec4.groupby(by = "restaurant name").sum().reset_index()
        # Sort values by cosine values in descending order
        rest_rec4 = rest_rec4.sort_values('similarity', ascending = False).reset_index(drop = True)

        # Print recommendations
        print('\n')
        rest_rec4.info()
        return rest_rec4.head(10)  
    
    # Data results from recommendation engine
    data = recommendation(rate, search1, rating1, search2, rating2, search3, rating3)

    x = data.iloc[:10,1].values
    x = x[::-1]
    y = data.iloc[:10,0].values
    y = y[::-1]
    
    # Results showed via horizontal bar plot
    data = [go.Bar(x=x,
                   y=y,
                   marker={'color':'#1E90FF'},
                   width = 0.5,
                   name='Restaurants',
                   orientation='h')]

    return {'data': data,
            'layout': go.Layout(margin={'l': 250, 'r': 20, 't': 40, 'b': 40})}

# Function outputs and parameters
@app.callback(Output('table','data'),
              [Input('rate','value'),
              Input('search1','value'),
              Input('rating1','value'),
              Input('search2','value'),
              Input('rating2','value'),
              Input('search3','value'),
              Input('rating3','value')])

# Table to display recommended restaurants' ratings and reviews
def table_d(rate,search1,rating1,search2,rating2,search3,rating3):

    # Create recommendation function
    def recommendation (standard, rest1, rating1, rest2, rating2, rest3, rating3):
        
        # Deciding which algorithm to use based on standard
        if standard == 1:
            algo = load(open('SVD_Model.pkl', 'rb'))
        else:
            algo = load(open('SVD_Model_Food.pkl', 'rb'))

            # Cosine distance between vectors calculation
        def cosine_distance(vector_a = np.array, vector_b = np.array):
            return cosine(vector_a, vector_b)

        # Retrieve vectors by restaurant name
        def get_vector_by_rest_name(rest_name, trained_model):
            rest_row_idx = trained_model.trainset._raw2inner_id_items[rest_name]
            return trained_model.qi[rest_row_idx]

        # Get vectors by restaurant name for three restaurants
        vector1 = get_vector_by_rest_name(rest1, algo)
        score1 = rating1
        vector2 = get_vector_by_rest_name(rest2, algo)
        score2 = rating2
        vector3 = get_vector_by_rest_name(rest3, algo)
        score3 = rating3

        ##############################################################################################################

        # Calculate cosine similarity for all three chosen restaurants' vectors against all other restaurant vectors
        similarity_table1 = []
        for rest_name in algo.trainset._raw2inner_id_items.keys():
            rest_vector = get_vector_by_rest_name(rest_name, algo)
            similarity_score = cosine_distance(vector1, rest_vector)
            similarity_table1.append((1-similarity_score, rest_name))

        # Convert similarity table into a data frame
        rest_rec1 = pd.DataFrame(similarity_table1, columns = ['similarity', 'restaurant name'])
        # Scale cosine score by rating
        rest_rec1['similarity'] = rest_rec1['similarity'] * score1
        # Sort data set to descending
        rest_rec1 = rest_rec1.sort_values('similarity', ascending = False)

        ##############################################################################################################

        # Calculate cosine similarity for all three chosen restaurants' vectors against all other restaurant vectors
        similarity_table2 = []
        for rest_name in algo.trainset._raw2inner_id_items.keys():
            rest_vector = get_vector_by_rest_name(rest_name, algo)
            similarity_score = cosine_distance(vector2, rest_vector)
            similarity_table2.append((1-similarity_score, rest_name))

        # Convert similarity table into a data frame
        rest_rec2 = pd.DataFrame(similarity_table2, columns = ['similarity', 'restaurant name'])
        # Scale cosine score by rating
        rest_rec2['similarity'] = rest_rec2['similarity'] * score2
        # Sort data set to descending
        rest_rec2 = rest_rec2.sort_values('similarity', ascending = False)

        ##############################################################################################################

        # Calculate cosine similarity for all three chosen restaurants' vectors against all other restaurant vectors
        similarity_table3 = []
        for rest_name in algo.trainset._raw2inner_id_items.keys():
            rest_vector = get_vector_by_rest_name(rest_name, algo)
            similarity_score = cosine_distance(vector3, rest_vector)
            similarity_table3.append((1-similarity_score, rest_name))

        # Convert similarity table into a data frame
        rest_rec3 = pd.DataFrame(similarity_table3, columns = ['similarity', 'restaurant name'])
        # Scale cosine score by rating
        rest_rec3['similarity'] = rest_rec3['similarity'] * score3
        # Sort data set to descending
        rest_rec3 = rest_rec3.sort_values('similarity', ascending = False).reset_index()

        ##############################################################################################################

        # Create a list of all data frames
        df_list = [rest_rec1, rest_rec2, rest_rec3]
        # Concatenate all data frames by axis 0
        rest_rec4 = pd.concat(df_list, axis = 0)
        # Remove all three chosen restaurants 
        rest_rec4 = rest_rec4.loc[(rest_rec4['restaurant name'] != rest1) & (rest_rec4['restaurant name'] != rest2) &
                                  (rest_rec4['restaurant name'] != rest3)].reset_index(drop = True)
        # Scale cosine score by duplicates
        rest_rec4 = rest_rec4.groupby(by = "restaurant name").sum().reset_index()
        # Sort values by cosine values in descending order
        rest_rec4 = rest_rec4.sort_values('similarity', ascending = False).reset_index(drop = True)

        # Print recommendations
        print('\n')
        rest_rec4.info()
        return rest_rec4.head(10)  
    
    # Data results from recommendation engine
    data = recommendation(rate, search1, rating1, search2, rating2, search3, rating3)
    
    y = data.iloc[:10,0].values
    
    # Ratings and review data are decided is the user shooses all or food standard
    if rate == 1:
        doc1 = pd.read_csv('Data/All.csv')
        doc1 = doc1.drop(['Unnamed: 0'], axis = 1)
    else:
        doc1 = pd.read_csv('Data/Food.csv')
        doc1 = doc1.drop(['Unnamed: 0'], axis = 1)
    
    # Generate the content for the ratings and reviews table
    doc_sf = doc1
    doc_sf.columns = ['Business','user_id','','Stars','new_text','topic']
    s1 = doc_sf.iloc[(doc_sf["Business"] == y[0]).values,[0,2,3]]
    s1 = s1.sort_values(by=['Stars'],ascending=False)

    s2 = doc_sf.iloc[(doc_sf["Business"] == y[1]).values,[0,2,3]]
    s2 = s2.sort_values(by=['Stars'],ascending=False)

    s3 = doc_sf.iloc[(doc_sf["Business"] == y[2]).values,[0,2,3]]
    s3 = s3.sort_values(by=['Stars'],ascending=False)

    s4 = doc_sf.iloc[(doc_sf["Business"] == y[3]).values,[0,2,3]]
    s4 = s4.sort_values(by=['Stars'],ascending=False)

    s5 = doc_sf.iloc[(doc_sf["Business"] == y[4]).values,[0,2,3]]
    s5 = s5.sort_values(by=['Stars'],ascending=False)
    
    s6 = doc_sf.iloc[(doc_sf["Business"] == y[5]).values,[0,2,3]]
    s6 = s6.sort_values(by=['Stars'],ascending=False)

    s7 = doc_sf.iloc[(doc_sf["Business"] == y[6]).values,[0,2,3]]
    s7 = s7.sort_values(by=['Stars'],ascending=False)

    s8 = doc_sf.iloc[(doc_sf["Business"] == y[7]).values,[0,2,3]]
    s8 = s8.sort_values(by=['Stars'],ascending=False)

    s9 = doc_sf.iloc[(doc_sf["Business"] == y[8]).values,[0,2,3]]
    s9 = s9.sort_values(by=['Stars'],ascending=False)

    s10 = doc_sf.iloc[(doc_sf["Business"] == y[9]).values,[0,2,3]]
    s10 = s10.sort_values(by=['Stars'],ascending=False)

    data = pd.concat([s1,s2,s3,s4,s5,s6,s7,s8,s9,s10], ignore_index=True)
    
    return data.to_dict('records')


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Feb/2021 15:46:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2021 15:46:14] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2021 15:46:14] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2021 15:46:14] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2021 15:46:15] "POST /_dash-update-component HTTP/1.1" 200 -





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant name  908 non-null    object 
 1   similarity       908 non-null    float64
 2   index            908 non-null    float64
dtypes: float64(2), object(1)
memory usage: 21.4+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant name  908 non-null    object 
 1   similarity       908 non-null    float64
 2   index            908 non-null    float64
dtypes: float64(2), object(1)
memory usage: 21.4+ KB


127.0.0.1 - - [14/Feb/2021 15:46:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2021 15:46:18] "POST /_dash-update-component HTTP/1.1" 200 -
